# Объявление инпунтов

In [1]:
import os
import pandas as pd
import numpy as np
from keras.models import Sequential, Model
from keras.layers import Dense, Conv2D, Flatten, Activation, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras import applications, optimizers
from pandas import DataFrame

# Установка const параметров для обработки

In [2]:
img_size = 224
batch_size = 32

# Создание генератора для обучения с аугментацией изображений

In [15]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.4,
    height_shift_range=0.4,
    shear_range=0.4,
    zoom_range=0.4,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
)

In [16]:
train_generator = train_datagen.flow_from_directory(
    '/Users/fanfurick/Downloads/plates/train',
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='binary',
)

Found 40 images belonging to 2 classes.


In [17]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    '/Users/fanfurick/Downloads/plates',
    classes=['test'],
    target_size=(img_size, img_size),
    batch_size=1,
    shuffle=False,
    class_mode='binary'
)

Found 744 images belonging to 1 classes.


# Загрузка модели

In [18]:
base_model = applications.InceptionResNetV2(weights='imagenet', 
                                                include_top=False, 
                                                input_shape=(img_size, img_size, 3))

# Работа с моделью

In [19]:
base_model.trainable = False
x = base_model.output

In [20]:
x = Flatten()(x)  # Преобразование выходов в одномерный массив
x = Dense(4096, activation="relu")(x)  # Полносвязный слой с активацией ReLU
x = Dropout(0.6)(x)  # Слой Dropout для регуляризации
predictions = Dense(1, activation="sigmoid")(x)  # Выходной слой с сигмоидной активацией
model = Model(inputs=base_model.input, outputs=predictions)

# Компиляция модели

In [21]:
model.compile(loss='binary_crossentropy', optimizer=optimizers.RMSprop(learning_rate=1e-5), metrics=['accuracy'])

In [30]:
model.fit(
        train_generator,
        steps_per_epoch=10,
        epochs=20,
        verbose=1) 

Epoch 1/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 85ms/step - accuracy: 0.8494 - loss: 0.3346 
Epoch 2/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 206ms/step - accuracy: 0.7500 - loss: 0.5140
Epoch 3/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 86ms/step - accuracy: 0.9231 - loss: 0.2504 
Epoch 4/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 86ms/step - accuracy: 0.8719 - loss: 0.3753 
Epoch 5/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 84ms/step - accuracy: 0.8750 - loss: 0.2771 
Epoch 6/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 193ms/step - accuracy: 0.8500 - loss: 0.4016
Epoch 7/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 186ms/step - accuracy: 0.9100 - loss: 0.2502
Epoch 8/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 84ms/step - accuracy: 0.9231 - loss: 0.2104 
Epoch 9/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 86ms/step - accuracy: 0.9231 - loss: 0.1822 
Epoch 10/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 87ms/step - accuracy: 0.8781 - loss: 0.3595 
Epoch 11/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 85ms/step - accuracy: 0.9006 - loss: 0.3284 
Epoch 12/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 190ms/step

2024-10-05 13:29:06.143148: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 85ms/step - accuracy: 0.9006 - loss: 0.1719 
Epoch 15/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 190ms/step - accuracy: 0.8200 - loss: 0.5691
Epoch 16/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 187ms/step - accuracy: 0.8625 - loss: 0.3776
Epoch 17/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 86ms/step - accuracy: 0.8238 - loss: 0.5208 
Epoch 18/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 193ms/step - accuracy: 0.8875 - loss: 0.3856
Epoch 19/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 196ms/step - accuracy: 0.9325 - loss: 0.2400
Epoch 20/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 188ms/step - accuracy: 0.8075 - loss: 0.5717


In [31]:
test_generator.reset()

# Получение результата

In [32]:
predict = model.predict(test_generator, steps=len(test_generator.filenames))
data_frame = {'id': [n for n in range(len(predict))], 
     'label': ['dirty' if x > 0.5 else 'cleaned' for x in predict]}
result_df = DataFrame(data=data_frame)
print(result_df['label'].value_counts())
result_df.to_csv('result.csv', index=False)

744/744 ━━━━━━━━━━━━━━━━━━━━ 87s 117ms/step
label
dirty      456
cleaned    288
Name: count, dtype: int64


![](score_81.png)